In [ ]:
import tensorflow as tf
# Clear previous models/graphs
tf.keras.backend.clear_session()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def load_data(file_path='data/data_for_modeling.csv'):
    df = pd.read_csv(file_path, parse_dates=['date'])
    df.set_index('date', inplace=True)
    return df[['unique_id', 'units']]


def create_multitask_sequences(data, window, w_h, m_h, s_h):
    """
    Generate sliding windows and future targets from a 1D array `data`.
    """
    X_list, y_w, y_m, y_s = [], [], [], []
    n = len(data)
    for i in range(n - window - s_h + 1):
        seq = data[i:i+window]
        fut = data[i+window:i+window+s_h]
        X_list.append(seq)
        y_w.append(fut[:w_h])
        y_m.append(fut[:m_h])
        y_s.append(fut)
    X = np.stack(X_list)[..., None]
    return X, np.stack(y_w), np.stack(y_m), np.stack(y_s)


def build_dataset_and_holdouts(df, window, season_horizon, month_horizon, week_horizon):
    """
    For each unique_id series, split off its last horizons as holdouts,
    but here we only return training sequences (no in-function validation).
    """
    X_list, yw_list, ym_list, ys_list = [], [], [], []
    for uid, group in df.groupby('unique_id'):
        series = group['units'].sort_index().values
        train_data = series
        if len(train_data) < window + season_horizon:
            continue
        X, yw, ym, ys = create_multitask_sequences(
            train_data, window, week_horizon, month_horizon, season_horizon
        )
        X_list.append(X)
        yw_list.append(yw)
        ym_list.append(ym)
        ys_list.append(ys)
    X_all = np.concatenate(X_list, axis=0)
    y_w_all = np.concatenate(yw_list, axis=0)
    y_m_all = np.concatenate(ym_list, axis=0)
    y_s_all = np.concatenate(ys_list, axis=0)
    return X_all, {'week': y_w_all, 'month': y_m_all, 'season': y_s_all}


def build_multihead_model(window, week_horizon, month_horizon, season_horizon,
                          lstm_units=50, dropout_rate=0.2):
    inp = Input(shape=(window, 1), name='input')
    x = LSTM(
        lstm_units,
        activation='tanh',
        dropout=dropout_rate,
        recurrent_dropout=dropout_rate,
        name='lstm'
    )(inp)
    x = Dropout(dropout_rate, name='dropout')(x)
    out_w = Dense(week_horizon, name='week')(x)
    out_m = Dense(month_horizon, name='month')(x)
    out_s = Dense(season_horizon, name='season')(x)
    model = Model(inp, [out_w, out_m, out_s], name='multitask_lstm_dropout')
    model.compile(
        optimizer='adam',
        loss={'week': 'mse', 'month': 'mse', 'season': 'mse'},
        loss_weights={'week': 1.0, 'month': 1.0, 'season': 1.0},
        metrics={'week': [tf.keras.metrics.MeanSquaredError(name='mse')],
                 'month': [tf.keras.metrics.MeanSquaredError(name='mse')],
                 'season': [tf.keras.metrics.MeanSquaredError(name='mse')]}
    )
    return model


if __name__=='__main__':
    # Load data and parameters
    df = load_data()
    window, w_h, m_h, s_h = 30, 7, 30, 90

    # Split unique_ids into train/validation groups
    all_ids = df['unique_id'].unique()
    train_ids, val_ids = train_test_split(all_ids, test_size=0.2, random_state=42)

    # Build training set: all sequences from train_ids
    X_train, y_train = build_dataset_and_holdouts(
        df[df['unique_id'].isin(train_ids)],
        window, s_h, m_h, w_h
    )

    # Build validation set: sequences from val_ids
    X_val, y_val = build_dataset_and_holdouts(
        df[df['unique_id'].isin(val_ids)],
        window, s_h, m_h, w_h
    )

    # Build and train model with real hold-out
    model = build_multihead_model(window, w_h, m_h, s_h)
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=2
    )

    # Plot train/validation loss
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Total Loss'); plt.legend(); plt.show()

    # Final evaluation on validation set
    val_metrics = model.evaluate(X_val, y_val, verbose=0, return_dict=True)
    print("\nValidation Metrics:", val_metrics)
    # Compute per-series season MSE if desired
    preds = model.predict(X_val)[2]
    season_mse = mean_squared_error(y_val['season'], preds)
    print(f"Overall Season MSE: {season_mse:.3f}")
    


Epoch 1/20


In [5]:
import tensorflow as tf
# Clear previous models/graphs
tf.keras.backend.clear_session()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error


def load_data(file_path='data/data_for_modeling.csv'):
    df = pd.read_csv(file_path, parse_dates=['date'])
    df.set_index('date', inplace=True)
    return df[['unique_id', 'units']]


def build_multihead_model(window, week_horizon, month_horizon, season_horizon,
                          lstm_units=50, dropout_rate=0.1):
    inp = Input(shape=(window,1), name='input')
    x = LSTM(lstm_units, activation='tanh',
             dropout=dropout_rate, recurrent_dropout=dropout_rate,
             name='lstm')(inp)
    x = Dropout(dropout_rate, name='dropout')(x)
    out_w = Dense(week_horizon, name='week')(x)
    out_m = Dense(month_horizon, name='month')(x)
    out_s = Dense(season_horizon, name='season')(x)
    model = Model(inp, [out_w,out_m,out_s], name='multitask_lstm_dropout')
    model.compile(
        optimizer='adam',
        loss={'week':'mse','month':'mse','season':'mse'},
        loss_weights={'week':2.0, 'month':1.0, 'season':0.5})
    return model


def create_multitask_sequences(data, window, w_h, m_h, s_h):
    """
    Generate sliding windows and future targets from a 1D array `data`.
    """
    X, y_w, y_m, y_s = [], [], [], []
    n = len(data)
    for i in range(n - window - s_h + 1):
        seq = data[i:i+window]
        fut = data[i+window:i+window+s_h]
        X.append(seq)
        y_w.append(fut[:w_h])
        y_m.append(fut[:m_h])
        y_s.append(fut)
    X = np.array(X)[...,None]
    return X, np.array(y_w), np.array(y_m), np.array(y_s)


def train_time_based(df, window, w_h, m_h, s_h):
    """
    Splits each series by reserving the last `s_h` points for validation,
    and the `window` points just before those for the validation input.
    Returns concatenated training and validation sets.
    """
    X_tr, yw_tr, ym_tr, ys_tr = [],[],[],[]
    X_val, yw_val, ym_val, ys_val = [],[],[],[]

    for uid, group in df.groupby('unique_id'):
        series = group['units'].sort_index().values
        if len(series) < window + s_h:
            continue  # skip too-short series
        # 1) validation target tail and its preceding window for input
        val_tail = series[-s_h:]                    # true future
        val_input = series[-s_h-window:-s_h]        # last window before tail
        # 2) training data excludes both val_tail and val_input
        train_data = series[:-s_h-window]
        # 3) build training sequences
        X_t, y_w_t, y_m_t, y_s_t = create_multitask_sequences(
            train_data, window, w_h, m_h, s_h
        )
        X_tr.append(X_t); yw_tr.append(y_w_t); ym_tr.append(y_m_t); ys_tr.append(y_s_t)
        # 4) pack validation single-sample per series
        X_val.append(val_input.reshape(1,window,1))
        yw_val.append(val_tail[:w_h].reshape(1,w_h))
        ym_val.append(val_tail[:m_h].reshape(1,m_h))
        ys_val.append(val_tail.reshape(1,s_h))

    # concatenate all series
    X_train = np.concatenate(X_tr,0)
    y_w_train = np.concatenate(yw_tr,0)
    y_m_train = np.concatenate(ym_tr,0)
    y_s_train = np.concatenate(ys_tr,0)
    X_valid = np.concatenate(X_val,0)
    y_w_valid = np.concatenate(yw_val,0)
    y_m_valid = np.concatenate(ym_val,0)
    y_s_valid = np.concatenate(ys_val,0)

    return (X_train, { 'week':y_w_train,'month':y_m_train,'season':y_s_train }),\
            (X_valid, { 'week':y_w_valid,'month':y_m_valid,'season':y_s_valid })


if __name__=='__main__':
    df = load_data()
    window, w_h, m_h, s_h = 30, 7, 30, 90

    (X_train, y_train), (X_val, y_val) = train_time_based(df, window, w_h, m_h, s_h)

    model = build_multihead_model(window, w_h, m_h, s_h)
    # training on train set only
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=2
    )


    # plot training loss
    plt.plot(history.history['loss'], label='train_loss')
    plt.title('Training Loss'); plt.legend(); plt.show()

    # evaluate on held-out tails
    print("Validation MSEs:")
    val_results = model.evaluate(X_val, y_val, verbose=0, return_dict=True)
    print(val_results)
    # or compute per-series MSE if needed
    preds = model.predict(X_val)
    season_mse = mean_squared_error(y_val['season'], preds[2])
    print(f"Overall Season MSE: {season_mse:.3f}")


Epoch 1/20
5472/5472 - 50s - 9ms/step - loss: 2994.7429 - month_loss: 862.8679 - season_loss: 858.7582 - val_loss: 740.4335 - val_month_loss: 234.1551 - val_season_loss: 205.1736 - val_week_loss: 200.9929 - week_loss: 851.1666
Epoch 2/20
5472/5472 - 46s - 8ms/step - loss: 2006.1699 - month_loss: 575.7144 - season_loss: 585.0865 - val_loss: 587.4216 - val_month_loss: 191.9827 - val_season_loss: 177.2074 - val_week_loss: 152.8823 - week_loss: 568.9123
Epoch 3/20


KeyboardInterrupt: 